In [1]:
import pandas as pd

In [2]:
import sqlite3
conn = sqlite3.connect('my_data.db')
c = conn.cursor()

Find the total amount of poster_qty paper ordered in the orders table

In [3]:
pd.read_sql('''select sum(poster_qty) from orders''',conn)

,sum(poster_qty)
0,723646.0


Find the total amount of standard_qty paper ordered in the orders table

In [4]:
pd.read_sql('''select sum(standard_qty) from orders''',conn)

,sum(standard_qty)
0,1938346.0


Find the total dollar amount of sales using the total_amt_usd in the orders table

In [5]:
pd.read_sql('''select sum(total_amt_usd) from orders''',conn)

,sum(total_amt_usd)
0,23141511.83


Find the total amount spent on standard_amt_usd and gloss_amt_usd paper for each order in the orders table. 
This should give a dollar amount for each order in the table.

In [7]:
pd.read_sql('''select standard_amt_usd+gloss_amt_usd from orders''',conn)

,standard_amt_usd+gloss_amt_usd
0,778.55
1,1255.19
2,776.18
3,958.24
4,756.13
...,...
6907,1545.40
6908,706.54
6909,783.90
6910,816.20


Find the standard_amt_usd per unit of standard_qty paper. Your solution should use both an aggregation 
and a mathematical operator

In [8]:
pd.read_sql('''select sum(standard_amt_usd)/sum(standard_qty) from orders''',conn)

,sum(standard_amt_usd)/sum(standard_qty)
0,4.99


When was the earliest order ever placed? You only need to return the date

In [12]:
pd.read_sql('''select min(occurred_at) from orders''',conn)

,min(occurred_at)
0,2013-12-04T04:22:44.000Z


When did the most recent (latest) web_event occur?

In [13]:
pd.read_sql('''select max(occurred_at) from orders''',conn)

,max(occurred_at)
0,2017-01-02T00:02:40.000Z


Find the mean (AVERAGE) amount spent per order on each paper type, as well as 
the mean amount of each paper type purchased per order. 
Your final answer should have 6 values - one for each paper type for the average number of sales, 
as well as the average amount

In [16]:
pd.read_sql('''select avg(standard_amt_usd),avg(gloss_amt_usd),avg(poster_amt_usd),
avg(standard_qty),avg(gloss_qty),avg(poster_qty) from orders''',conn)

,avg(standard_amt_usd),avg(gloss_amt_usd),avg(poster_amt_usd),avg(standard_qty),avg(gloss_qty),avg(poster_qty)
0,1399.355692,1098.54742,850.116539,318.440283,172.000848,125.567586


Which account (by name) placed the earliest order? Your solution should have the account name and the date of the order.

In [28]:
pd.read_sql('''select accounts.name,orders.occurred_at from orders 
inner join accounts on orders.account_id=accounts.id
order by orders.occurred_at limit 1''',conn)

,name,occurred_at
0,DISH Network,2013-12-04T04:22:44.000Z


Find the total sales in usd for each account. You should include two columns - the total sales for each company's orders in usd 
and the company name

In [32]:
pd.read_sql('''select accounts.name,sum(orders.total_amt_usd) from orders
inner join accounts on accounts.id=orders.account_id
group by orders.account_id''',conn)

,name,sum(orders.total_amt_usd)
0,Walmart,124014.87
1,Exxon Mobil,2734.59
2,Apple,20767.15
3,Berkshire Hathaway,7474.32
4,McKesson,15184.46
...,...,...
345,KKR,217473.85
346,Oneok,2844.46
347,Newmont Mining,9618.81
348,PPL,112176.03


Via what channel did the most recent (latest) web_event occur, which account was associated with this web_event? 
Your query should return only three values - the date, channel, and account name

In [37]:
pd.read_sql('''select max(web_events.occurred_at),web_events.channel,accounts.name from web_events
inner join accounts on web_events.account_id=accounts.id''',conn)

,max(web_events.occurred_at),channel,name
0,2017-01-01T23:51:09.000Z,organic,Molina Healthcare


Find the total number of times each type of channel from the web_events was used. 
Your final table should have two columns - the channel and the number of times the channel was used

In [39]:
pd.read_sql('''select web_events.channel, count(1) from web_events group by 1''',conn)

,channel,count(1)
0,adwords,906
1,banner,476
2,direct,5298
3,facebook,967
4,organic,952
5,twitter,474


Who was the Sales Rep associated with the earliest web_event?

In [43]:
pd.read_sql('''select sales_reps.name,min(web_events.occurred_at) from web_events
inner join accounts on accounts.id=web_events.account_id
inner join sales_reps on sales_reps.id=accounts.sales_rep_id''',conn)

,name,min(web_events.occurred_at)
0,Dorotha Seawell,2013-12-04T04:18:29.000Z


Who was the primary contact associated with the earliest web_event?

In [44]:
pd.read_sql('''select accounts.primary_poc,min(web_events.occurred_at) from web_events
inner join accounts on accounts.id=web_events.account_id''',conn)

,primary_poc,min(web_events.occurred_at)
0,Leana Hawker,2013-12-04T04:18:29.000Z


What was the smallest order placed by each account in terms of total usd. 
Provide only two columns - the account name and the total usd. Order from smallest dollar amounts to largest

In [48]:
pd.read_sql('''select accounts.name,min(orders.total_amt_usd) from accounts
inner join orders on orders.account_id=accounts.id
group by 1''',conn)

,name,min(orders.total_amt_usd)
0,3M,673.78
1,ADP,121.80
2,AECOM,199.60
3,AES,2277.74
4,AIG,902.30
...,...,...
345,World Fuel Services,711.23
346,Xcel Energy,381.85
347,Xerox,74.85
348,Yum Brands,81.20


Find the number of sales reps in each region. 
Your final table should have two columns - the region and the number of sales_reps. Order from fewest reps to most reps.

In [50]:
pd.read_sql('''select region.name,count(sales_reps.id) from region
join sales_reps on region.id=sales_reps.region_id
group by region.id''',conn)

,name,count(sales_reps.id)
0,Northeast,21
1,Midwest,9
2,Southeast,10
3,West,10


For each account, determine the average amount of each type of paper they purchased across their orders. 
Your result should have four columns - one for the account name and one for the average quantity purchased 
for each of the paper types for each account

In [54]:
pd.read_sql('''select accounts.name,avg(standard_qty),avg(poster_qty),avg(gloss_qty) from orders
join accounts on orders.account_id=accounts.id
group by 1''',conn)

,name,avg(standard_qty),avg(poster_qty),avg(gloss_qty)
0,3M,313.392857,120.730769,290.111111
1,ADP,434.244898,75.897959,69.846154
2,AECOM,409.250000,21.428571,20.833333
3,AES,254.666667,147.500000,304.000000
4,AIG,300.500000,108.000000,349.000000
...,...,...,...,...
345,World Fuel Services,208.000000,30.000000,206.750000
346,Xcel Energy,417.166667,13.250000,188.400000
347,Xerox,355.666667,117.666667,38.000000
348,Yum Brands,421.100000,36.200000,52.727273


For each account, determine the average amount spent per order on each paper type. 
Your result should have four columns - one for the account name and one for the average amount spent on each paper type

In [56]:
pd.read_sql('''select accounts.name,avg(standard_amt_usd),avg(poster_amt_usd),avg(gloss_amt_usd) from orders
join accounts on orders.account_id=accounts.id
group by 1''',conn)

,name,avg(standard_amt_usd),avg(poster_amt_usd),avg(gloss_amt_usd)
0,3M,1563.830357,910.310000,2095.327500
1,ADP,1769.620333,503.304667,453.394667
2,AECOM,1815.251111,135.333333,104.027778
3,AES,1270.786667,798.466667,2276.960000
4,AIG,1499.495000,876.960000,2614.010000
...,...,...,...,...
345,World Fuel Services,1037.920000,121.800000,1548.557500
346,Xcel Energy,2081.661667,71.726667,1175.930000
347,Xerox,1331.082500,716.590000,142.310000
348,Yum Brands,1400.859333,195.962667,289.613333


Determine the number of times a particular channel was used in the web_events table for each sales rep. 
Your final table should have three columns - the name of the sales rep, the channel, and the number of occurrences. 
Order your table with the highest number of occurrences first

In [60]:
pd.read_sql('''select sales_reps.name,web_events.channel,count(2) from sales_reps
join accounts on sales_reps.id=accounts.sales_rep_id
join web_events on accounts.id=web_events.account_id
group by 2''',conn)

,name,channel,count(2)
0,Samuel Racine,adwords,906
1,Samuel Racine,banner,476
2,Samuel Racine,direct,5298
3,Samuel Racine,facebook,967
4,Samuel Racine,organic,952
5,Samuel Racine,twitter,474


Determine the number of times a particular channel was used in the web_events table for each region. 
Your final table should have three columns - the region name, the channel, and the number of occurrences. 
Order your table with the highest number of occurrences first

In [69]:
pd.read_sql('''select region.name,web_events.channel,count(2) from region
join sales_reps on region.id=sales_reps.region_id
join accounts on sales_reps.id=accounts.sales_rep_id
join web_events on web_events.account_id=accounts.id
group by region.id
order by count(2) desc''',conn)

,name,channel,count(2)
0,Northeast,adwords,3061
1,Southeast,adwords,2638
2,West,adwords,2205
3,Midwest,banner,1169


Have any sales reps worked on more than one account?

In [73]:
pd.read_sql('''select sales_rep_id,id,count(1),count(2) from accounts
group by 1,2
order by count(2) desc''',conn)

,sales_rep_id,id,count(1),count(2)
0,321500,1001,1,1
1,321500,1211,1,1
2,321500,1421,1,1
3,321500,1631,1,1
4,321500,1661,1,1
...,...,...,...,...
346,321990,3991,1,1
347,321990,4091,1,1
348,321990,4191,1,1
349,321990,4251,1,1


How many of the sales reps have more than 5 accounts that they manage?

In [77]:
pd.read_sql('''select sales_rep_id,id,count(2) from accounts
group by 1
having count(2)>5
order by count(2) desc''',conn)

,sales_rep_id,id,count(2)
0,321970,3571,15
1,321980,3581,11
2,321960,3561,11
3,321850,2591,11
4,321840,3361,11
5,321830,2571,11
6,321820,2561,11
7,321800,2541,11
8,321940,3541,10
9,321930,3531,10


How many accounts have more than 20 orders?

In [82]:
pd.read_sql('''select accounts.id,count(orders.id) from accounts
join orders on accounts.id=orders.account_id
group by 1
having count(orders.id)>20
order by count(orders.id) desc''',conn)

,id,count(orders.id)
0,3411,71
1,2591,68
2,1561,68
3,2181,67
4,4151,66
...,...,...
115,2571,22
116,2191,22
117,4171,21
118,2841,21


Which account has the most orders?

In [86]:
pd.read_sql('''select accounts.name,count(orders.id) from accounts
join orders on accounts.id=orders.account_id
group by 1
order by count(orders.id) desc
limit 1''',conn)

,name,count(orders.id)
0,Leucadia National,71


Which accounts spent more than 30,000 usd total across all orders?

In [89]:
pd.read_sql('''select accounts.name,sum(total_amt_usd) from orders
join accounts on accounts.id=orders.account_id
group by 1
having sum(total_amt_usd)>30000''',conn)

,name,sum(total_amt_usd)
0,3M,127945.10
1,ADP,163579.18
2,AT&T,51647.76
3,Abbott Laboratories,96819.92
4,Aetna,237781.30
...,...,...
199,Wells Fargo,200174.66
200,Western Digital,269155.34
201,Western Refining,108579.84
202,Whirlpool,32965.13


Which accounts spent less than 1,000 usd total across all orders?

In [90]:
pd.read_sql('''select accounts.name,sum(total_amt_usd) from orders
join accounts on accounts.id=orders.account_id
group by 1
having sum(total_amt_usd)<1000''',conn)

,name,sum(total_amt_usd)
0,Delta Air Lines,859.64
1,Level 3 Communications,881.73
2,Nike,390.25


Which account has spent the most with us?

In [91]:
pd.read_sql('''select accounts.name,sum(total_amt_usd) from orders
join accounts on accounts.id=orders.account_id
group by 1
order by sum(total_amt_usd) desc
limit 1''',conn)

,name,sum(total_amt_usd)
0,EOG Resources,382873.3


Which account has spent the least with us?

In [92]:
pd.read_sql('''select accounts.name,sum(total_amt_usd) from orders
join accounts on accounts.id=orders.account_id
group by 1
order by sum(total_amt_usd)
limit 1''',conn)

,name,sum(total_amt_usd)
0,Nike,390.25


Which accounts used facebook as a channel to contact customers more than 6 times?

In [105]:
pd.read_sql('''select accounts.name,web_events.channel,count(channel) from accounts
join web_events on accounts.id=web_events.account_id
where web_events.channel='facebook'
group by 1
having count(channel)>6
''',conn)

,name,channel,count(channel)
0,ADP,facebook,9
1,Aetna,facebook,10
2,Apple,facebook,9
3,Arrow Electronics,facebook,9
4,AutoNation,facebook,14
5,Avis Budget Group,facebook,7
6,Best Buy,facebook,7
7,BlackRock,facebook,10
8,CST Brands,facebook,9
9,Cameron International,facebook,9


Which account used facebook most as a channel?

In [106]:
pd.read_sql('''select accounts.name,web_events.channel,count(channel) from accounts
join web_events on accounts.id=web_events.account_id
where web_events.channel='facebook'
group by 1
order by count(channel) desc
limit 1''',conn)

,name,channel,count(channel)
0,Gilead Sciences,facebook,16


Which channel was most frequently used by most accounts?

In [110]:
pd.read_sql('''select web_events.channel,count(channel) from accounts
join web_events on accounts.id=web_events.account_id
group by channel
order by count(channel) desc
limit 1
''',conn)

,channel,count(channel)
0,direct,5298


Which channel was most frequently used by most accounts? (including account name)

In [115]:
pd.read_sql('''select accounts.name,web_events.channel,count(channel) from accounts
join web_events on accounts.id=web_events.account_id
group by channel,accounts.id
order by count(channel) desc
limit 1
''',conn)

,name,channel,count(channel)
0,Leucadia National,direct,52


Find the sales in terms of total dollars for all orders in each year, ordered from greatest to least

In [135]:
pd.read_sql('''select strftime('%Y',occurred_at),sum(total_amt_usd) from orders
group by 1
order by 2 desc
''',conn)

,"strftime('%Y',occurred_at)",sum(total_amt_usd)
0,2016,12864917.92
1,2015,5752004.94
2,2014,4069106.54
3,2013,377331.00
4,2017,78151.43


Which month did Parch & Posey have the greatest sales in terms of total dollars?

In [137]:
pd.read_sql('''select strftime('%m',occurred_at),sum(total_amt_usd) from orders
where occurred_at between '2014-01-01' AND '2017-01-01'
group by 1
order by 2 desc
''',conn)

,"strftime('%m',occurred_at)",sum(total_amt_usd)
0,12,2752080.98
1,10,2427505.97
2,11,2390033.75
3,09,2017216.88
4,07,1978731.15
5,08,1918107.22
6,06,1871118.52
7,03,1659987.88
8,04,1562037.74
9,05,1537082.23


Which year did Parch & Posey have the greatest sales in terms of total number of orders?

In [140]:
pd.read_sql('''select strftime('%Y',occurred_at),count(orders.id) from orders
where occurred_at between '2014-01-01' AND '2017-01-01'
group by 1
order by 2 desc
''',conn)

,"strftime('%Y',occurred_at)",count(orders.id)
0,2016,3757
1,2015,1725
2,2014,1306


Which month did Parch & Posey have the greatest sales in terms of total number of orders? 

In [141]:
pd.read_sql('''select strftime('%m',occurred_at),count(orders.id) from orders
where occurred_at between '2014-01-01' AND '2017-01-01'
group by 1
order by 2 desc
''',conn)

,"strftime('%m',occurred_at)",count(orders.id)
0,12,783
1,11,713
2,10,675
3,08,603
4,09,602
5,07,571
6,06,527
7,05,518
8,03,482
9,04,472


In which month of which year did Walmart spend the most on gloss paper in terms of dollars?

In [145]:
pd.read_sql('''select strftime('%m',occurred_at),sum(gloss_amt_usd) from orders
join accounts on orders.account_id=accounts.id
where (occurred_at between '2014-01-01' AND '2017-01-01') and accounts.name='Walmart'
group by 1
order by 2 desc
''',conn)

,"strftime('%m',occurred_at)",sum(gloss_amt_usd)
0,05,9257.64
1,11,9250.15
2,12,5774.79
3,01,5070.73
4,04,4875.99
5,03,4711.21
6,02,4673.76
7,09,4673.76
8,08,4531.45
9,07,4254.32


Write a query to display for each order, the account ID, total amount of the order, and the level of the order - ‘Large’ or ’Small’
- depending on if the order is $3000 or more, or smaller than $3000

In [152]:
pd.read_sql('''select account_id,total_amt_usd,
case when total_amt_usd>3000 then 'large'
else 'small' 
end as cat
from orders''',conn)

,account_id,total_amt_usd,cat
0,1001,973.43,small
1,1001,1718.03,small
2,1001,776.18,small
3,1001,958.24,small
4,1001,983.49,small
...,...,...,...
6907,4501,2024.48,small
6908,4501,1486.06,small
6909,4501,1449.74,small
6910,4501,1473.92,small


Write a query to display the number of orders in each of three categories, based on the total number of items in each order. 
The three categories are: 'At Least 2000', 'Between 1000 and 2000' and 'Less than 1000'

In [154]:
pd.read_sql('''select 
case
when total >= 2000 then "at least 2000"
when total >=1000 and total<2000 then "between 1000 and 20000"
else "less than 1000"
end as cat, count(1)
from orders
group by 1
''',conn)

,cat,count(1)
0,at least 2000,70
1,between 1000 and 20000,511
2,less than 1000,6331


We would like to understand 3 different levels of customers based on the amount associated with their purchases. 
The top level includes anyone with a Lifetime Value (total sales of all orders) greater than 200,000 usd. 
The second level is between 200,000 and 100,000 usd. 
The lowest level is anyone under 100,000 usd. 
Provide a table that includes the level associated with each account. 
You should provide the account name, the total sales of all orders for the customer, and the level. 
Order with the top spending customers listed first.

In [155]:
pd.read_sql('''select 
accounts.name,
case
when sum(total_amt_usd)>= 200000 then "first"
when sum(total_amt_usd) >=100000 and sum(total_amt_usd)<200000 then "second"
else "third"
end as cat, sum(total_amt_usd)
from orders join
accounts on accounts.id=orders.account_id
group by 1
''',conn)

,name,cat,sum(total_amt_usd)
0,3M,second,127945.10
1,ADP,second,163579.18
2,AECOM,third,18491.51
3,AES,third,13038.64
4,AIG,third,9980.93
...,...,...,...
345,World Fuel Services,third,10833.11
346,Xcel Energy,third,19975.91
347,Xerox,third,8759.93
348,Yum Brands,third,28296.53


We would now like to perform a similar calculation to the first, 
but we want to obtain the total amount spent by customers only in 2016 and 2017. 
Keep the same levels as in the previous question. Order with the top spending customers listed first.

In [157]:
pd.read_sql('''select 
accounts.name,
case
when sum(total_amt_usd)>= 200000 then "first"
when sum(total_amt_usd) >=100000 and sum(total_amt_usd)<200000 then "second"
else "third"
end as cat, sum(total_amt_usd)
from orders join
accounts on accounts.id=orders.account_id
where strftime('%Y',orders.occurred_at) in ('2016','2017')
group by 1
''',conn)

,name,cat,sum(total_amt_usd)
0,3M,third,70895.01
1,ADP,third,37456.04
2,AECOM,third,18491.51
3,AES,third,13038.64
4,AIG,third,9980.93
...,...,...,...
317,World Fuel Services,third,10833.11
318,Xcel Energy,third,19975.91
319,Xerox,third,8759.93
320,Yum Brands,third,28296.53


We would like to identify top performing sales reps, which are sales reps associated with more than 200 orders. 
Create a table with the sales rep name, the total number of orders, 
and a column with top or not depending on if they have more than 200 orders. Place the top sales people first in your final table.

In [162]:
pd.read_sql('''select sales_reps.name,count(orders.id),
case
when count(orders.id)>200 then "top"
else "not"
end as top from orders 
join accounts on orders.account_id=accounts.id
join sales_reps on accounts.sales_rep_id=sales_reps.id
group by 1
order by 2 desc''',conn)

,name,count(orders.id),top
0,Earlie Schleusner,335,top
1,Vernita Plump,299,top
2,Tia Amato,267,top
3,Georgianna Chisholm,256,top
4,Moon Torian,250,top
5,Nelle Meaux,241,top
6,Maren Musto,224,top
7,Dorotha Seawell,208,top
8,Charles Bidwell,205,top
9,Maryanna Fiorentino,204,top


The previous didn't account for the middle, nor the dollar amount associated with the sales. 
Management decides they want to see these characteristics represented as well. 
We would like to identify top performing sales reps, which are sales reps associated with more than 200 orders 
or more than 750000 in total sales. The middle group has any rep with more than 150 orders or 500000 in sales. 
Create a table with the sales rep name, the total number of orders, total sales across all orders, 
and a column with top, middle, or low depending on this criteria. 
Place the top sales people based on dollar amount of sales first in your final table. 
You might see a few upset sales people by this criteria!

In [164]:
pd.read_sql('''select sales_reps.name,count(orders.id),
case
when count(orders.id)>200 or sum(total_amt_usd)>750000 then "top"
when (count(orders.id)>150 and count(orders.id)<=200) or sum(total_amt_usd)>500000 then 'middle'
else "low"
end as level from orders 
join accounts on orders.account_id=accounts.id
join sales_reps on accounts.sales_rep_id=sales_reps.id
group by 1
order by 2 desc''',conn)

,name,count(orders.id),level
0,Earlie Schleusner,335,top
1,Vernita Plump,299,top
2,Tia Amato,267,top
3,Georgianna Chisholm,256,top
4,Moon Torian,250,top
5,Nelle Meaux,241,top
6,Maren Musto,224,top
7,Dorotha Seawell,208,top
8,Charles Bidwell,205,top
9,Maryanna Fiorentino,204,top
